In [1]:

from langchain_community.document_loaders.csv_loader import CSVLoader 
file_path = "customer.csv" # Need not change the path if you are forking the Repo
loader = CSVLoader(file_path=file_path,encoding="utf-8",metadata_columns=["flags","category","intent","response"])
data = loader.load() # Loading the document objects
print("Documents loaded :)")
# data

Documents loaded :)


In [2]:
from langchain_huggingface import HuggingFaceEmbeddings  # Using hugging face embeddings for the embedding model
from tqdm import TqdmWarning   # Just using this to ignore warnings from tqdm
import warnings
warnings.filterwarnings("ignore",category=TqdmWarning)
warnings.filterwarnings("ignore",category=UserWarning)
warnings.filterwarnings("ignore",category=FutureWarning)

print("Loading embedding model....")
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
print("Loaded and up!")
data_list = [] # A list to store the Page content (Columns: ["instruction"])
metadata = [] # A list to store the metadata (Columns: ["flags","category","intent","response"])
count = 0 # A variable for indexing purpose

for i in data:
    data_list.append(data[count].page_content)
    metadata.append(data[count].metadata)
    count += 1
print("Embedding starting....")
embeded_val = embedding.embed_documents(data_list) # Embedding the data_list using the embedding model
print("Values Embedded!")


Loading embedding model....
Loaded and up!
Embedding starting....
Values Embedded!


In [3]:
from langchain_chroma import Chroma # Using Chroma DB as our vector database
db1 = Chroma.from_texts(texts=data_list,embedding=embedding,metadatas=metadata) # Storing the embedded values to the database

In [4]:
query = "i am not able to update my delivery address"
# query = "I want to cancel my order"
# query = "what is the cancellation charges for my order?"
# query = "i want to delete my account"

embedding_vector = embedding.embed_query("instruction: "+query) # Converting the text query into vectors for semantic searching
docs_and_scores = db1.similarity_search_by_vector_with_relevance_scores(embedding_vector) # Searching the database for the top 4 documents (default k value is 4)
docs_and_scores
for i in range(0,4):
    print(docs_and_scores[i])

(Document(page_content='instruction: I have a problem updating my delivery address', metadata={'category': 'SHIPPING', 'flags': 'BL', 'intent': 'change_shipping_address', 'response': "I'm sorry to hear that you're having trouble updating your delivery address. Could you please provide more details about the issue you are experiencing? This will help me assist you better.", 'row': 2162, 'source': 'customer.csv'}), 0.058140963315963745)
(Document(page_content='instruction: I am trying to update my delivery address', metadata={'category': 'SHIPPING', 'flags': 'BL', 'intent': 'change_shipping_address', 'response': 'No worries! Updating your delivery address is a simple process. You can do it by logging into your account on our website and going to the "Manage Addresses" section. From there, you can edit your current address or add a new one. If you need any help, feel free to reach out to our customer service team.', 'row': 2845, 'source': 'customer.csv'}), 0.06622816622257233)
(Document(p

In [5]:
from langchain_community.llms.ollama import Ollama

llm = Ollama(model="llama3",system="You are a efficient customer care agent working at Amazon. Analyze the 'Realtime customer query', 'predefined query' and the 'predefined response' and return a new response for the Customer (keep the responses short (within 30 words) and to the point as it is a call). If the 'Realtime customer query' and the 'predefined query' don't match, reject the query politely")
chunks = llm.stream(input="query: " + docs_and_scores[0][0].page_content[13:] +" predefined response: " + docs_and_scores[0][0].metadata['response']) # Here Stream was used so that we don't have to wait for the whole response to be generated from the model
for chunk in chunks:
    print(chunk,end='',flush=True)

New Response:

"I apologize for the inconvenience, Amazon allows only one active default shipping address. Please ensure there are no duplicate addresses or try updating from Your Account > Shipping Settings. If the issue persists, I'll be happy to further assist you."